## Import modules

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download("stopwords")
import re
from nltk.corpus import stopwords

## Loading the dataset

In [ ]:
df = pd.read_csv('spam.csv', encoding="ISO-8859-1")
df.head()

In [ ]:
# rename v2: messages, v1: label
df = df.rename(columns={'v2': 'messages', 'v1': 'label'})

## Preprocessing the dataset

In [ ]:
df.isnull().sum()

In [ ]:
STOPWORDS = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^0-9a-zA-Z]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = " ".join(word for word in text.split() if word not in STOPWORDS)
    return text

In [ ]:
df['clean_text'] = df['messages'].apply(clean_text)
df.head()

## Input Split

In [ ]:
X = df['clean_text']
y = df['label']

## Model Training

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

def classify(model, X, y):
    # train test split
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True, stratify=y)
    # model training
    pipeline_model = Pipeline([('vect', CountVectorizer()),
                              ('tfidf', TfidfTransformer()),
                              ('clf', model)])
    pipeline_model.fit(x_train, y_train)
    
    print('Accuracy:', pipeline_model.score(x_test, y_test)*100)
    
    # cv_score = cross_val_score(model, X, y, cv=5)
    # print("CV Score:", np.mean(cv_score)*100)
    y_pred = pipeline_model.predict(x_test)
    print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
classify(model, X, y)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
classify(model, X, y)

In [ ]:
from sklearn.svm import SVC
model = SVC(C=3)
classify(model, X, y)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
classify(model, X, y)